In [ ]:
##########################################
###
### test the python code in https://github.com/gioda/FeARLesS/tree/main
###
##########################################

In [1]:
from skimage import measure
import pandas as pd
from skimage.filters import threshold_otsu, rank
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pyshtools as pysh
from skimage.io import imread
from skimage.io import imsave
from skimage import filters
from skimage import morphology
from pyclesperanto_prototype import imshow
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt

In [3]:
import napari
from napari.utils import nbscreenshot

In [4]:
cle.available_device_names()

['Apple M1 Max']

In [5]:
# For 3D processing, powerful graphics
# processing units might be necessary
cle.select_device('TX')

/Users/jingkui.wang/miniforge3/envs/image_env/lib/python3.9/site-packages/pyclesperanto_prototype/_tier0/_device.py:77: UserWarning: No OpenCL device found with TX in their name. Using Apple M1 Max instead.
  warnings.warn(f"No OpenCL device found with {name} in their name. Using {device.name} instead.")


<Apple M1 Max on Platform: Apple (2 refs)>

In [6]:
### import function https://github.com/gioda/FeARLesS/blob/main/Fearless/utils.py
import pyshtools
import gc
import numpy as np
import os
from vedo import printc, spher2cart, probePoints
#from vedo import Points
import shutil
from sys import exit

In [7]:
from vedo import spher2cart
from vedo import printc, spher2cart, probePoints

In [8]:
def samplePoints(vol, expo, N, radiusDiscretisation):
    """Compute sample points."""
    pos = vol.center()
    rmax = vol.diagonalSize()/2

    samplePoints = []
    for th in np.linspace(0, np.pi, N, endpoint=False):
        for ph in np.linspace(0, 2*np.pi, N, endpoint=False):

            # compute sample points
            p = spher2cart(rmax, th, ph)
            # making discretization more dense away from the center
            p_tmp = p / (radiusDiscretisation-1)**expo
            for j in range(radiusDiscretisation):
                SP = pos + p_tmp * (j**expo)
                samplePoints.append(SP)

    del vol
    return np.array(samplePoints)

In [9]:
def confirm(message):
    """
    Ask user to enter Y or N (case-insensitive).

    :return: True if the answer is Y.
    :rtype: bool
    """
    answer = ""
    while answer not in ["y", "n"]:
        answer = input(message).lower()
    return answer == "y"


def pathExists(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
        printc("Directory ", path, " Created ", c='green')
    else:
        printc("Directory ", path, " already exists", c='red')
        if confirm("Should I delete the folder and create a new one [Y/N]? "):
            shutil.rmtree(path)
            os.makedirs(path, exist_ok=True)
            printc("Directory ", path, " Created ", c='green')
        else:
            exit()


def voxelIntensity(vol, expo, N, radiusDiscretisation):
    """Compute voxel intensities."""
    pos = vol.center()
    rmax = vol.diagonalSize()/2

    scalars = []

    for th in np.linspace(0, np.pi, N, endpoint=False):
        for ph in np.linspace(0, 2*np.pi, N, endpoint=False):

            # compute sample points
            p = spher2cart(rmax, th, ph)
            samplePointsTmp = []
            
            # making discretization more dense away from the center
            p_tmp = p / (radiusDiscretisation-1)**expo
            for j in range(radiusDiscretisation):
                SP = pos + p_tmp * (j**expo)
                samplePointsTmp.append(SP)

            # compute intensities
            pb = probePoints(vol, samplePointsTmp)

            del samplePointsTmp

            # making the intensities growing outside the volume according to the gradient
            scalarsTmp = pb.getPointArray()
            nonz = np.nonzero(scalarsTmp)[0]
            if len(nonz) > 2:
                lastNoZeroId = nonz[-1]  # find the last value != 0
                secondlastNoZeroId = nonz[-2]
                # find the last value != 0
                lastNoZero = scalarsTmp[lastNoZeroId]
                secondlastNoZero = scalarsTmp[secondlastNoZeroId]
                dx = lastNoZero - secondlastNoZero

            for i in range(lastNoZeroId+1, len(scalarsTmp)):
                scalarsTmp[i] = scalarsTmp[i-1] + dx
            scalars.append(scalarsTmp.tolist())

            del pb, scalarsTmp

    del vol
    gc.collect()

    # return allIntensitiesMatrix
    return np.array(scalars).reshape((N * N, radiusDiscretisation))


def forwardTransformation(matrixOfIntensities, N, lmax):

    ##############################################

    coeff = matrixOfIntensities

    ##############################################
    # SPHARNM
    allClm = np.zeros((matrixOfIntensities.shape[1], 2, lmax, lmax))
    for j in range(allClm.shape[0]):
        formattedcoeff = np.reshape(coeff[:, j], (N, N))
        SH = pyshtools.SHGrid.from_array(formattedcoeff)
        clm = SH.expand()

        allClm[j, :, :, :] = clm.to_array(lmax=lmax - 1)

    del formattedcoeff, clm, matrixOfIntensities

    return allClm


def inverseTransformations(allClm, allIntensitiesShape, N, lmax):
    """Make inverse SPHARM."""
    from scipy.interpolate import griddata

    aSH_recoMatrix = np.zeros((allIntensitiesShape[0], allIntensitiesShape[1]))

    for j in range(allClm.shape[0]):
        # inverse SPHARM coefficients
        clmCoeffs = pyshtools.SHCoeffs.from_array(allClm[j, :, :, :])
        SH_reco = clmCoeffs.expand(lmax=lmax - 1)
        # grid_reco.plot()
        aSH_reco = SH_reco.to_array()

        ##############################
        pts1 = []
        ll = []
        for ii, long in enumerate(np.linspace(0, 360, num=aSH_reco.shape[1], endpoint=True)):
            for jj, lat in enumerate(np.linspace(90, -90, num=aSH_reco.shape[0], endpoint=True)):
                th = np.deg2rad(90 - lat)
                ph = np.deg2rad(long)
                p = spher2cart(aSH_reco[jj][ii], th, ph)
                pts1.append(p)
                ll.append((lat, long))

        radii = aSH_reco.T.ravel()

        # make a finer grid
        n = N * 1j
        l_min, l_max = np.array(ll).min(axis=0), np.array(ll).max(axis=0)
        grid = np.mgrid[l_max[0]:l_min[0]:n, l_min[1]:l_max[1]:n]
        grid_x, grid_y = grid
        agrid_reco_finer = griddata(ll, radii, (grid_x, grid_y), method='cubic')
        ##############################

        formatted_aSH_reco = np.reshape(agrid_reco_finer, (N * N))

        aSH_recoMatrix[:, j] = formatted_aSH_reco

    del formatted_aSH_reco, agrid_reco_finer, grid_x, grid_y, grid

    return aSH_recoMatrix




In [ ]:
###########################################################################
## STEP 1: 
## run the script from https://github.com/gioda/FeARLesS/blob/main/Fearless/pureSPharm.py
## for limb_noFlank meshes
###########################################################################

In [31]:

from sys import argv, exit
from scipy.interpolate import griddata
import numpy as np

from vedo import printc, load, spher2cart, mag, ProgressBar, Points, write
from vedo import recoSurface
from vedo import *
#from utils import pathExists

import pyshtools
#############################################################

In [11]:
def computeCLM(mesh, rmax, N, x0):
    """Compute CLM."""
    # cast rays from the center and find intersections
    agrid, pts = [], []
    for th in np.linspace(0, np.pi, N, endpoint=False):
        lats = []
        for ph in np.linspace(0, 2*np.pi, N, endpoint=False):
            p = spher2cart(rmax, th, ph)
            intersections = mesh.intersectWithLine(x0, x0 + p)
            if len(intersections):
                value = mag(intersections[0]-x0)
                lats.append(value)
                pts.append(intersections[0])
            else:
                lats.append(rmax)
                # lats.append(0)
                pts.append(p)
        agrid.append(lats)
    agrid = np.array(agrid)
    
    grid = pyshtools.SHGrid.from_array(agrid)
    clm = grid.expand()
    # grid_reco = clm.expand(lmax=lmax)  # cut "high frequency" components

    return clm

In [12]:
lmax = 20
N = 500          # number of grid intervals on the unit sphere
rmax = 1400
x0 = [0, 0, 0]  # set object at this position
xLimb = [-200, 0, 200]
cutOrigin = [150, 0, 0]
deg_fit = 6

In [13]:
DataPath = '/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/limbs/limbs-noFlank/'
path_results = 'res/' \
    'pure_spharm' + '-lmax' + str(lmax) + '-N' + \
    str(N) + '-deg_fit' + str(deg_fit) + '/'


pathExists(path_results)

printc('lmax =', lmax, 'N =', N, 'deg_fit =', deg_fit, c='y')


Directory  res/pure_spharm-lmax20-N500-deg_fit6/  already exists


Should I delete the folder and create a new one [Y/N]?  Y


Directory  res/pure_spharm-lmax20-N500-deg_fit6/  Created 
lmax = 20 N = 500 deg_fit = 6


In [14]:
DataPath

'/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/limbs/limbs-noFlank/'

In [15]:
printc("loading limbs ...", c='y')
limbs = load(DataPath + '*.vtk')

totLimbs = len(limbs)

printc('tot # limbs --> ', totLimbs, c='y')

loading limbs ...
tot # limbs -->  69


In [16]:
len(limbs)
limbs[58].filename

'/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/limbs/limbs-noFlank/Limb_279.vtk'

In [17]:
################################
# finding time from limbs' file names
Tall = []
for j in range(totLimbs):
    filename = limbs[j].filename.split('/')[-1].split('.')[0]
    if '_' in filename:
        number_part = filename.split('_')[1]
    else:
        number_part = filename.split('.')[0]
    try:
        Tall.append(float(number_part))
    except ValueError:
        print(f"Could not extract a valid number from filename: {filename}")
        continue

Tall = np.array(Tall)
printc('Tall \n', Tall, c='y')

Treal = np.arange(Tall[0], Tall[-1]+1, dtype=int)

NumRecLimbs = len(Treal)


Tall 
 [249. 250. 251. 254. 257. 257. 257. 257. 259. 260. 260. 261. 262. 263.
 264. 264. 265. 265. 265. 265. 265. 265. 266. 266. 266. 267. 268. 268.
 269. 270. 270. 270. 270. 271. 271. 271. 271. 272. 272. 272. 273. 273.
 274. 274. 274. 274. 274. 275. 275. 275. 275. 275. 276. 276. 276. 277.
 277. 278. 279. 283. 284. 285. 287. 288. 288. 288. 289. 290. 290.]


In [18]:
Tall

array([249., 250., 251., 254., 257., 257., 257., 257., 259., 260., 260.,
       261., 262., 263., 264., 264., 265., 265., 265., 265., 265., 265.,
       266., 266., 266., 267., 268., 268., 269., 270., 270., 270., 270.,
       271., 271., 271., 271., 272., 272., 272., 273., 273., 274., 274.,
       274., 274., 274., 275., 275., 275., 275., 275., 276., 276., 276.,
       277., 277., 278., 279., 283., 284., 285., 287., 288., 288., 288.,
       289., 290., 290.])

In [19]:
totLimbs
limbs;

In [20]:
Mclm = []
pb = ProgressBar(0, totLimbs, c=2)
print(pb)

In [21]:
for j in pb.range():
    Mtmp = []
    clmAllTmp = []
    #
    filename = limbs[j].filename.split('/')[-1].split('.')[0]
    if '_' in filename:
        try:
            number_part = filename.split('_')[1]
        except IndexError:
            print(f"Unexpected filename format with underscore: {filename}")
            continue
    else:
        number_part = filename.split('.')[0]

    try:
        Mtmp.append(float(number_part))
    except ValueError:
        print(f"Could not extract a valid number from filename: {filename}")
        continue
    #

    clmTmp = computeCLM(limbs[j].pos(xLimb), rmax, N, x0)

    clmAllTmp.append(clmTmp.to_array())
    Mtmp.append(np.asarray(clmAllTmp))

    Mclm.append(Mtmp)

    pb.print('clm...')


 ─────────────────────── 1% clm... ────────────────────── 3%     ETA: 11m57s (0.1 it/s) clm... ────────────────────── 4%                             ETA: 11m47s (0.1 it/s) clm... ────────────────────── 6%                             ETA: 11m43s (0.1 it/s) clm... ━───────────────────── 7%                             ETA: 12m1s (0.1 it/s) clm... ━───────────────────── 9%                           ETA: 13m53s (0.1 it/s) clm... ━───────────────────── 10%                             ETA: 14m2s (0.1 it/s) clm... ━━──────────────────── 12%                           ETA: 13m45s (0.1 it/s) clm... ━━──────────────────── 13%                             ETA: 13m55s (0.1 it/s) clm... ━━──────────────────── 14%                             ETA: 14m8s (0.1 it/s) clm... ━━━─────────────────── 16%                           ETA: 14m14s (0.1 it/s) clm... ━━━─────────────────── 17%                             ETA: 14m21s (0.1 it/s) clm... ━━━─────────────────── 19%                             ETA: 14m21s (

In [22]:
pb.range();
totLimbs

69

In [23]:
CLM = sorted(Mclm, key=lambda tup: tup[0])

CLMtot = []
for j in range(len(CLM)):
    CLMtot.append(CLM[j][1])


In [24]:
CLMtot = np.asarray(CLMtot)
CLMtot = np.squeeze(CLMtot, axis=1)

filename = 'clm_N' + str(N) + '_lmax'+str(lmax) + '.npy'
printc('saving --> ', filename, c='g')
np.save(path_results + filename, CLMtot)


saving -->  clm_N500_lmax20.npy


In [25]:
# CLMtot = np.load(path_results + 'clm_N500_lmax50.npy')

clmSpline = np.zeros(
    shape=(NumRecLimbs, CLMtot.shape[1], CLMtot.shape[2], CLMtot.shape[3]))
print('clmSpline.shape', clmSpline.shape)


clmSpline.shape (42, 2, 250, 250)


In [26]:
maxt = np.max(Treal)
mint = np.min(Treal)
xnew = np.linspace(mint, maxt, NumRecLimbs)
range_m = np.linspace(
    0, NumRecLimbs, num=clmSpline.shape[0], endpoint=False).astype(int)


In [27]:
pb = ProgressBar(0, CLMtot.shape[1], c=2)
for ll in pb.range():
    for k in range(CLMtot.shape[2]):
        for j in range(CLMtot.shape[3]):

            spl = np.poly1d(np.polyfit(
                np.array(Tall), np.array(CLMtot[:,  ll, k, j]), deg_fit))
            ynew = spl(xnew)

            for m in range(clmSpline.shape[0]):
                clmSpline[m, ll, k, j] = ynew[range_m[m]]

    pb.print('splines...')

 ━━━━━━━━━━━─────────── 50% splines... ━━━━━━━━━━━━━━━━━━━━━━         elapsed: 6s (0.3 it/s)        


In [28]:
pb = ProgressBar(0, clmSpline.shape[0], c=2)

In [29]:
#from vedo import reconstruct_surface

In [36]:
for t in pb.range():

    clmCoeffs = pyshtools.SHCoeffs.from_array(clmSpline[t])

    grid_reco = clmCoeffs.expand(lmax=lmax)
    # grid_reco.plot()

    ##############################################
    agrid_reco = grid_reco.to_array()

    ll = []
    for i, long in enumerate(np.linspace(0, 360, num=agrid_reco.shape[1], endpoint=True)):
        for j, lat in enumerate(np.linspace(90, -90, num=agrid_reco.shape[0], endpoint=True)):
            th = np.deg2rad(90 - lat)
            ph = np.deg2rad(long)
            p = spher2cart(agrid_reco[j][i], th, ph)
            ll.append((lat, long))

    radii = agrid_reco.T.ravel()
    n = 2*N * 1j
    lnmin, lnmax = np.array(ll).min(axis=0), np.array(ll).max(axis=0)
    grid = np.mgrid[lnmax[0]:lnmin[0]:n, lnmin[1]:lnmax[1]:n]
    grid_x, grid_y = grid
    agrid_reco_finer = griddata(ll, radii, (grid_x, grid_y), method='cubic')

    pts2 = []
    for i, long in enumerate(np.linspace(0, 360, num=agrid_reco_finer.shape[1], endpoint=False)):
        for j, lat in enumerate(np.linspace(90, -90, num=agrid_reco_finer.shape[0], endpoint=True)):
            th = np.deg2rad(90 - lat)
            ph = np.deg2rad(long)
            p = spher2cart(agrid_reco_finer[j][i], th, ph)
            pts2.append(p)

    #mesh2 = Points(pts2, r=20, c="r", alpha=1)
    
    mesh2 = Points(pts2, r=20, c="r", alpha=1)
    
    mesh2.clean(0.005)
    surfTmp = recoSurface(mesh2.cutWithPlane(origin=[-30, 0, 0]),
                          dims=100
                          )
    #surfTmp = reconstruct_surface(mesh2.cutWithPlane(origin=[-30, 0, 0]),
    #                      dims=100
    #                      )
    
    #mesh2.clean()
    #surfTmp = Points.reconstruct_surface(mesh2.cut_with_plane(origin=[-30, 0, 0]),
    #                      dims=100
    #                      )

    surf = surfTmp.extractLargestRegion().clone()
    surf.smooth()

    # #Reconstruction
    write(mesh2, path_results + 'Points_Limb-rec_' +
          str(Treal[t]) + '.vtk', binary=False)
    write(surf, path_results + 'Limb-rec_' +
          str(Treal[t]) + '.vtk', binary=False)

    pb.print('rec ...')

 ─────────────────────── 2% rec ... ────────────────────── 5%       ETA: 147m20s (0.0 it/s) rec ... ━───────────────────── 7%                               ETA: 97m21s (0.0 it/s) rec ... ━───────────────────── 10%                             ETA: 72m16s (0.0 it/s) rec ... ━━──────────────────── 12%                             ETA: 57m11s (0.0 it/s) rec ... ━━──────────────────── 14%                             ETA: 47m6s (0.0 it/s) rec ... ━━━─────────────────── 17%                             ETA: 39m53s (0.0 it/s) rec ... ━━━─────────────────── 19%                             ETA: 34m28s (0.0 it/s) rec ... ━━━━────────────────── 21%                             ETA: 30m15s (0.0 it/s) rec ... ━━━━━───────────────── 24%                             ETA: 26m51s (0.0 it/s) rec ... ━━━━━───────────────── 26%                             ETA: 24m4s (0.0 it/s) rec ... ━━━━━━──────────────── 29%                             ETA: 21m44s (0.0 it/s) rec ... ━━━━━━──────────────── 31%               

In [11]:
###########################################################################
## STEP 2: 
## python makeVoxel.py
## for limbs+flank
###########################################################################
"""Convert a vtk mesh from the limb-data files into voxel data."""

In [24]:
#from utils import pathExists
from vedo import ProgressBar, load, printc, volumeFromMesh, write
import numpy as np

In [26]:
DataPath = 'S-BIAD441/limbs/limbs+flank/'
limbs = load(DataPath + '*.vtk')
totLimbs = len(limbs)
printc("limbs # ", totLimbs, invert=1)

limbs #  69


In [27]:
sampleSize = (100, 100, 100)

path_results = 'res/TIF-signedDist_sampleSize' + \
    str(sampleSize[0]) + '/'

pathExists(path_results)

Directory  res/TIF-signedDist_sampleSize100/  already exists


Should I delete the folder and create a new one [Y/N]?  Y


Directory  res/TIF-signedDist_sampleSize100/  Created 


In [28]:
# needed to have all the normals pointing in the same direction
noMirror = [4, 5, 6, 8, 9, 13, 14, 16, 17, 18, 19, 20, 22, 23, 25, 26, 27, 29, 30, 31, 33, 34,
            35, 37, 38, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 52, 53, 55, 56, 59, 63, 64, 67, 68]

# collecting imgBounds of all limbs
allBounds = np.empty(shape=(6, totLimbs))
pb = ProgressBar(0, totLimbs, c=1)
for j in pb.range():
    allBounds[:, j] = limbs[j].GetBounds()
    pb.print('all bounds ...')

# finding max imgBounds
allBoundsTmp = []
pb = ProgressBar(0, allBounds.shape[0], c=1)
for j in pb.range():
    if allBounds[j, 0] > 0:
        allBoundsTmp.append(np.max(allBounds[j, :]))
    else:
        allBoundsTmp.append(np.min(allBounds[j, :]))
    pb.print('allBoundsTmp ... ')

# making imgBounds a bit bigger, just in case
imgBOunds = [x * 1.2 for x in allBoundsTmp]

 ─────────────────────── 1% all bounds ... ────────────────────── 3%             elapsed: 0s (613.1 it/s)         ────────────────────── 4%                               elapsed: 0s (618.3 it/s)         ────────────────────── 6%                               elapsed: 0s (535.3 it/s)         ━───────────────────── 7%                               elapsed: 0s (541.9 it/s)         ━───────────────────── 9%                               elapsed: 0s (399.8 it/s)         ━───────────────────── 10%                               elapsed: 0s (408.8 it/s)         ━━──────────────────── 12%                               elapsed: 0s (446.5 it/s)         ━━──────────────────── 13%                               elapsed: 0s (481.6 it/s)         ━━──────────────────── 14%                               elapsed: 0s (517.1 it/s)         ━━━─────────────────── 16%                               elapsed: 0s (549.2 it/s)         ━━━─────────────────── 17%                               elapsed: 0s (581.9 it/s

In [29]:
pb = ProgressBar(0, totLimbs, c=1)
pb.range()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68])

In [30]:
j = 0
#time = limbs[j].filename.split('.')[0].split('/')[-1].split('_')[1]
limbs[j].filename.split('.')[0]

'S-BIAD441/limbs/limbs+flank/Limb_249'

In [31]:
for j in pb.range():

    time = limbs[j].filename.split('.')[0].split('/')[-1].split('_')[1]

    if limbs[j].filename.split('.')[0].split('/')[-1].split('_')[-1] == 'clean-newAlignement':
        outputName = path_results + 'ReferenceShape_' + time + '_0.vti'
    else:
        outputName = path_results + 'ReferenceShape_' + time + '_' + \
            limbs[j].filename.split('.')[0].split(
                '/')[-1].split('_')[-1] + '.vti'

    if j in noMirror:
        vol = volumeFromMesh(limbs[j],
                             dims=sampleSize,
                             bounds=imgBOunds,
                             signed=True,
                             negate=True,  # invert sign
                             )
        write(vol, outputName)
    else:
        vol = volumeFromMesh(limbs[j],
                             dims=sampleSize,
                             bounds=imgBOunds,
                             signed=True,
                             negate=False,  # invert sign
                             )
        write(vol, outputName)

    pb.print('making voxel data ...')

 ─────────────────────── 1% making voxel data ... ────────────────────── 3%                     ETA: 27m38s (0.0 it/s) making voxel data ... ────────────────────── 4%                                           ETA: 26m47s (0.0 it/s) making voxel data ... ────────────────────── 6%                                           ETA: 25m2s (0.0 it/s) making voxel data ... ━───────────────────── 7%                                           ETA: 23m25s (0.0 it/s) making voxel data ... ━───────────────────── 9%                                           ETA: 22m42s (0.0 it/s) making voxel data ... ━───────────────────── 10%                                           ETA: 23m10s (0.0 it/s) making voxel data ... ━━──────────────────── 12%                                           ETA: 21m47s (0.0 it/s) making voxel data ... ━━──────────────────── 13%                                           ETA: 21m16s (0.0 it/s) making voxel data ... ━━──────────────────── 14%                                        

In [ ]:
##########################################################################
# STEP 3:
# run the computeAllIntesities.py with heart volumetric data for limb data
##########################################################################

In [32]:
from vedo import printc, load, ProgressBar
#from utils import pathExists, voxelIntensity
import numpy as np

In [33]:
radiusDiscretisation = 50
N = 250
FFTexpansion = radiusDiscretisation
expo = 1.0

In [34]:
DataPath = 'res/TIF-signedDist_sampleSize100/'
path_results = 'res/' \
    'allIntensities-sampleSize100-' + \
    'radiusDiscretisation-' + \
    str(radiusDiscretisation) + '-N-' + str(N) + '/'

pathExists(path_results)

Directory  res/allIntensities-sampleSize100-radiusDiscretisation-50-N-250/  Created 


In [35]:
limbs = [2490, 2500, 2510, 2540, 2570, 2571, 2573, 2574, 2590, 2600, 2601, 2610, 2620, 2631, 2640, 2642, 2650, 2651, 2652, 2653, 2654, 2655, 2660,
         2662, 2663, 2671, 2681, 2682, 2690, 2700, 2701, 2703, 2704, 2710, 2711, 2712, 2714, 2720, 2721, 2722, 2731, 2732, 2740, 2741, 2742, 2745,
         2746, 2750, 2751, 2752, 2754, 2755, 2760, 2761, 2762, 2771, 2772, 2780, 2790, 2831, 2840, 2850, 2870, 2880, 2881, 2882, 2890, 2901, 2902]
t = [249, 250, 251, 254, 257, 257, 257, 257, 259, 260, 260, 261, 262, 263, 264, 264, 265, 265, 265, 265, 265, 265, 266, 266, 266, 267,
     268, 268, 269, 270, 270, 270, 270, 271, 271, 271, 271, 272, 272, 272, 273, 273, 274, 274, 274, 274, 274, 275, 275, 275, 275, 275, 276, 276, 276,
     277, 277, 278, 279, 283, 284, 285, 287, 288, 288, 288, 289, 290, 290]

Treal = np.arange(t[0], t[-1]+1, dtype=int)
newTotLimbs = len(Treal)

totlimbs = len(limbs)
print(totlimbs, 'limbs!! \n')


69 limbs!! 



In [42]:
Treal

array([249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261,
       262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274,
       275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287,
       288, 289, 290])

In [44]:
limbs = load(DataPath + '*.vti')

In [45]:
j = 0
#DataPath + 'ReferenceShape_' + str(limbs[j])[0:-1] + '_' + str(limbs[j])[-1] + '.vti'
time = limbs[j].filename.split('.')[0].split('/')[-1].split('_')[1]
'ReferenceShape_' + time + '_' + \
            limbs[j].filename.split('.')[0].split(
                '/')[-1].split('_')[-1] + '.vti'

'ReferenceShape_249_249.vti'

In [46]:
time

'249'

In [41]:
str(limbs[j])[-1]

'0'

In [49]:
limbs[j].filename.split('.')[0].split('/')[-1].split('_')[-1]
time

'249'

In [50]:
pb = ProgressBar(0, totlimbs, c=5)
for j in pb.range():

    ##############################################
    # loading files
    time = limbs[j].filename.split('.')[0].split('/')[-1].split('_')[1]
    
    
    vol = load(DataPath + 'ReferenceShape_' + time + '_' + limbs[j].filename.split('.')[0].split('/')[-1].split('_')[-1] + '.vti')
    
    ##############################################
    # computing voxel intensities
    allIntensities = voxelIntensity(vol, expo, N, radiusDiscretisation)

    name = 'allIntensities_' + 'ReferenceShape_' + time + '_' + limbs[j].filename.split('.')[0].split('/')[-1].split('_')[-1]
    
    np.save(path_results + name, allIntensities)
    printc('allIntensities', name, 'saved!', c='g')

    allIntensitiesShape = allIntensities.shape
    np.save(path_results + 'allIntensitiesShape', allIntensitiesShape)

    pb.print()
    

allIntensities allIntensities_ReferenceShape_249_249 saved!
 ─────────────────────── 1% allIntensities allIntensities_ReferenceShape_250_250 saved!
 ────────────────────── 3% ETA: 18m40s (0.1 it/s) allIntensities allIntensities_ReferenceShape_251_251 saved!
 ────────────────────── 4%                       ETA: 18m31s (0.1 it/s) allIntensities allIntensities_ReferenceShape_254_254 saved!
 ────────────────────── 6%                       ETA: 18m15s (0.1 it/s) allIntensities allIntensities_ReferenceShape_257_1 saved!
 ━───────────────────── 7%                       ETA: 17m59s (0.1 it/s) allIntensities allIntensities_ReferenceShape_257_257 saved!
 ━───────────────────── 9%                       ETA: 17m44s (0.1 it/s) allIntensities allIntensities_ReferenceShape_257_3 saved!
 ━───────────────────── 10%                       ETA: 17m28s (0.1 it/s) allIntensities allIntensities_ReferenceShape_257_4 saved!
 ━━──────────────────── 12%                       ETA: 17m14s (0.1 it/s) allIntensities

In [ ]:
#### run the computaeAllINtensities.py with heart volumetric data
input_path = '/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/hearts/'
output_path = "/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/hearts/output/reconstructed_shapes/"
intensity_path = "/Users/jingkui.wang/workspace/imp/image_analysis/S-BIAD441/hearts/output/voxel_intensities/"
os.makedirs(output_path, exist_ok=True)
os.makedirs(intensity_path, exist_ok=True)

In [ ]:
volume_files = [f for f in os.listdir(input_path) if f.endswith('.vti')]

In [ ]:
len(volume_files)

In [ ]:
vol_file = os.path.join(input_path, volume_files[20])
vol = load(vol_file)
vol
allIntensities = voxelIntensity(vol, expo, N, radiusDiscretisation)
allIntensities
allIntensities[:1]
# Forward Transformations (Splines and SPHARM)
Clm = forwardTransformation(allIntensities, N, lmax = 20)
Clm.shape
allIntensities.shape
coeff = allIntensities
lmax = 20
# SPHARNM
allClm = np.zeros((allIntensities.shape[1], 2, lmax, lmax))
allClm.shape
range(allClm.shape[0])
formattedcoeff = np.reshape(coeff[:, 1], (N, N))
formattedcoeff.shape
SH = pyshtools.SHGrid.from_array(formattedcoeff)
SH
#xx = pyshtools.expand.SHExpandDH(formattedcoeff, sampling=2)
clm = SH.expand()
clm
clm.plot_spectrum(show=False)

#power_per_l = pyshtools.spectralanalysis.spectrum(clm)
#degrees = np.arange(SH.shape[1])

fig, ax = plt.subplots(1, 1)
ax.plot(degrees, power_per_l)
ax.set(yscale='log', xscale='log', xlabel='Spherical harmonic degree', ylabel='Power')
ax.grid()


In [ ]:
volume_files = [f for f in os.listdir(input_path) if f.endswith('.vti')]
len(volume_files)

totVols = len(volume_files)
pb = ProgressBar(0, totVols, c=5)
pb.range()

for j in pb.range():

    ##############################################
    # loading files
    volume_file = volume_files[j]
    vol_file = os.path.join(input_path, vol_file)
    vol = load(vol_file)
    ##############################################
    # computing voxel intensities
    allIntensities = voxelIntensity(vol, expo, N, radiusDiscretisation)

    allIntensitiesShape = allIntensities.shape
    
    name = f"intensities_{os.path.splitext(volume_file)[0]}.npy"
    np.save(os.path.join(intensity_path, name), allIntensities)
    print(f"Voxel intensities saved: {name}")

    shape_file = f"shape_{os.path.splitext(volume_file)[0]}.npy"
    np.save(os.path.join(intensity_path, shape_file), allIntensities.shape)
    print(f"Voxel intensity shape saved: {shape_file}")
        
    pb.print()
    
    

In [51]:
###########################################################################
## STEP 4: 
## python morphing.py
## for limbs+flank
###########################################################################
"""Convert a vtk mesh from the limb-data files into voxel data."""

'Convert a vtk mesh from the limb-data files into voxel data.'

In [52]:
import numpy as np
from vedo import printc, ProgressBar, load, Points, write, interpolateToVolume
#from utils import pathExists, forwardTransformation, samplePoints, inverseTransformations


In [88]:
# PARAMETERS
sampleSize = 100
radiusDiscretisation = 50
N = 250
lmax = 50
expo = 1.0
deg_fit = 6

printc('\n sampleSize', sampleSize, '- radiusDiscretisation',
       radiusDiscretisation, '- N', N, '- lmax', lmax,
       c='green')


 sampleSize 100 - radiusDiscretisation 50 - N 250 - lmax 50


In [89]:
path = 'res/' \
    'allIntensities-sampleSize100-' + \
    'radiusDiscretisation-' + \
    str(radiusDiscretisation) + '-N-' + str(N) + '/'
path_resultsCLM = 'res/' \
    '/CLM/'\
    'morphing_sampleSize' + str(sampleSize) + \
    '-radDisc' + str(radiusDiscretisation) + '-N' + str(N) + '-degFit' + \
    str(deg_fit) + '-lmax'+str(lmax) + '/'
path_results = 'res/' \
    'morphing_sampleSize' \
    + str(sampleSize) + \
    '-radDisc' + str(radiusDiscretisation) + '-N' + str(N) + '-degFit' + \
    str(deg_fit) + '-lmax'+str(lmax) + '/'

pathExists(path_resultsCLM)
pathExists(path_results)

Directory  res//CLM/morphing_sampleSize100-radDisc50-N250-degFit6-lmax50/  already exists


Should I delete the folder and create a new one [Y/N]?  Y


Directory  res//CLM/morphing_sampleSize100-radDisc50-N250-degFit6-lmax50/  Created 
Directory  res/morphing_sampleSize100-radDisc50-N250-degFit6-lmax50/  already exists


Should I delete the folder and create a new one [Y/N]?  Y


Directory  res/morphing_sampleSize100-radDisc50-N250-degFit6-lmax50/  Created 


In [90]:
limbs = [2490, 2500, 2510, 2540, 2570, 2571, 2573, 2574, 2590, 2600, 2601, 2610, 2620, 2631, 2640, 2642, 2650, 2651, 2652, 2653, 2654, 2655, 2660,
         2662, 2663, 2671, 2681, 2682, 2690, 2700, 2701, 2703, 2704, 2710, 2711, 2712, 2714, 2720, 2721, 2722, 2731, 2732, 2740, 2741, 2742, 2745,
         2746, 2750, 2751, 2752, 2754, 2755, 2760, 2761, 2762, 2771, 2772, 2780, 2790, 2831, 2840, 2850, 2870, 2880, 2881, 2882, 2890, 2901, 2902]
t = [249, 250, 251, 254, 257, 257, 257, 257, 259, 260, 260, 261, 262, 263, 264, 264, 265, 265, 265, 265, 265, 265, 266, 266, 266, 267,
     268, 268, 269, 270, 270, 270, 270, 271, 271, 271, 271, 272, 272, 272, 273, 273, 274, 274, 274, 274, 274, 275, 275, 275, 275, 275, 276, 276, 276,
     277, 277, 278, 279, 283, 284, 285, 287, 288, 288, 288, 289, 290, 290]

Treal = np.arange(t[0], t[-1]+1, dtype=int)
newTotLimbs = len(Treal)

totlimbs = len(limbs)
print(totlimbs, 'limbs!! \n')


# matrix with all Clm of all limbs
allClmMatrix = np.zeros((totlimbs, radiusDiscretisation, 2, lmax, lmax), dtype=np.float32, order='C')

69 limbs!! 



In [91]:
pb = ProgressBar(0, totlimbs, c=5)  # it should loop over totLimbs!!
limbs = [f for f in os.listdir(path) if f.endswith('.npy')]
limbs;

In [92]:
path

'res/allIntensities-sampleSize100-radiusDiscretisation-50-N-250/'

In [93]:
limbs[0]

'allIntensities_ReferenceShape_277_1.npy'

In [94]:
printc('\n starting...', invert=1)
for j in pb.range():
    ##############################################
    # loading voxel intensities
    allIntensities = np.load(path + limbs[j])
    
    allIntensitiesShape = allIntensities.shape

    ##############################################
    # Forward Transformations (Splines and SPHARM)
    Clm = forwardTransformation(allIntensities, N, lmax)

    allClmMatrix[j, :, :, :, :] = Clm

    del Clm, allIntensities
    pb.print('Forward Transformations (splines and SPHARM) ...')

np.save(path_resultsCLM + 'allClmMatrix', allClmMatrix)
# np.save(pathOutClm + 'samplePointsCoord', samplePointsCoord)
# np.save(pathOutClm + 'allIntensities', allIntensities)
np.save(path_resultsCLM+'allIntensitiesShape', allIntensitiesShape)
printc('\n allIntensitiesShape.npy  \n', c='green')


 starting...
 ─────────────────────── 1% Forward Transformations (splines and SPHARM) ... ────────────────────── 3%                                               ETA: 22m10s (0.1 it/s) Forward Transformations (splines and SPHARM) ... ────────────────────── 4%                                                                       ETA: 14m35s (0.1 it/s) Forward Transformations (splines and SPHARM) ... ────────────────────── 6%                                                                       ETA: 10m48s (0.1 it/s) Forward Transformations (splines and SPHARM) ... ━───────────────────── 7%                                                                       ETA: 8m31s (0.1 it/s) Forward Transformations (splines and SPHARM) ... ━───────────────────── 9%                                                                     ETA: 7m0s (0.1 it/s) Forward Transformations (splines and SPHARM) ... ━───────────────────── 10%                                                                     ETA

In [95]:
limbs[0]
limbs[0].split('.')[0].split('_')[-2]
#time = limbs[0].split('.')[0].split('/')[-1].split('_')[1]
#time

'277'

In [97]:
allClmMatrix;

In [100]:
t = []
nb_t = len(limbs)

for j in range(nb_t):
    t.append(int(limbs[j].split('.')[0].split('_')[-2]))

t[0],limbs[0]

(277, 'allIntensities_ReferenceShape_277_1.npy')

In [109]:
t;
Treal = np.arange(t[0], t[-1]+1, dtype=int)
newTotLimbs = len(Treal)

totlimbs = len(limbs)
print(totlimbs, 'limbs!! \n')
newTotLimbs

69 limbs!! 



0

In [111]:
t = [249, 250, 251, 254, 257, 257, 257, 257, 259, 260, 260, 261, 262, 263, 264, 264, 265, 265, 265, 265, 265, 265, 266, 266, 266, 267,
     268, 268, 269, 270, 270, 270, 270, 271, 271, 271, 271, 272, 272, 272, 273, 273, 274, 274, 274, 274, 274, 275, 275, 275, 275, 275, 276, 276, 276,
     277, 277, 278, 279, 283, 284, 285, 287, 288, 288, 288, 289, 290, 290]

Treal = np.arange(t[0], t[-1]+1, dtype=int)
newTotLimbs = len(Treal)
newTotLimbs

42

In [112]:
t[0], t[-1]+1

(249, 291)

In [102]:
##########
# splining
##########
maxt = np.max(t)
mint = np.min(t)

xnew = np.linspace(mint, maxt, newTotLimbs)
allClmSpline = np.zeros(shape=(newTotLimbs, radiusDiscretisation,
                        2, allClmMatrix.shape[3], allClmMatrix.shape[4]))
# new clm from splines
range_m = np.linspace(0, newTotLimbs, num=allClmSpline.shape[0], endpoint=False).astype(int)


In [105]:
maxt, mint, allClmMatrix.shape

(290, 249, (69, 50, 2, 50, 50))

In [106]:
pb = ProgressBar(0, allClmMatrix.shape[1], c=5)
for f in pb.range():
    for ll in range(allClmMatrix.shape[2]):
        for k in range(allClmMatrix.shape[3]):
            for j in range(allClmMatrix.shape[4]):
                spl = np.poly1d(np.polyfit(np.array(t), np.array(
                    allClmMatrix[:, f, ll, k, j]), deg_fit))
                ynew = spl(xnew)

                for m in range(allClmSpline.shape[0]):
                    allClmSpline[m, f, ll, k, j] = ynew[range_m[m]]

    pb.print('splining ...')

print('allClmSpline.shape', allClmSpline.shape)

#del allClmMatrix


np.save(path_resultsCLM + 'allClmSpline', allClmSpline)
printc('\n allClmSpline.npy \n', c='green')

 ─────────────────────── 2% splining ... ────────────────────── 4%           ETA: 10s (4.8 it/s) splining ... ────────────────────── 6%                               ETA: 10s (4.8 it/s) splining ... ━───────────────────── 8%                               ETA: 9s (4.9 it/s) splining ... ━───────────────────── 10%                               ETA: 9s (4.9 it/s) splining ... ━━──────────────────── 12%                               ETA: 9s (5.0 it/s) splining ... ━━──────────────────── 14%                               ETA: 9s (5.0 it/s) splining ... ━━━─────────────────── 16%                               ETA: 8s (5.0 it/s) splining ... ━━━─────────────────── 18%                               ETA: 8s (5.0 it/s) splining ... ━━━━────────────────── 20%                               ETA: 8s (5.0 it/s) splining ... ━━━━────────────────── 22%                               ETA: 8s (5.0 it/s) splining ... ━━━━━───────────────── 24%                               ETA: 8s (5.0 it/s) splining ... ━

In [108]:
allClmSpline.shape[0]

0

In [84]:
##############################################
# vol parameters

pathTmp = 'res/TIF-signedDist_sampleSize' + str(sampleSize) + '/'


volTmp = load(pathTmp + 'ReferenceShape_290_2.vti')
pos = volTmp.center()
rmax = volTmp.diagonalSize()/2
volBounds = np.array(volTmp.GetBounds())
np.save(path_resultsCLM + 'volBounds', volBounds)
printc('\n volBounds.npy saved\n', c='green')

samplePoints = samplePoints(volTmp, expo, N, radiusDiscretisation)
del volTmp


 volBounds.npy saved



In [107]:
path_results
allClmSpline

array([], shape=(0, 50, 2, 50, 50), dtype=float64)

In [85]:
##############################################
# inverse Transformations
pb = ProgressBar(0, allClmSpline.shape[0], c=5)

for t in pb.range():
    ##############################################
    # Inverse Transformations
    ##############################################
    inverse_Matrix = inverseTransformations(allClmSpline[t, :, :, :, :], allIntensitiesShape, N, lmax)

    intensitiesreshape = np.reshape(
        inverse_Matrix, inverse_Matrix.shape[0] * inverse_Matrix.shape[1])

    del inverse_Matrix

    # ##############################
    # #   interpolateToImageData
    voxBin = 100  # * radiusDiscretisation
    apts = Points(samplePoints).addPointArray(intensitiesreshape, 'scals')
    volume = interpolateToVolume(apts, kernel='shepard', radius=(
        rmax / 20), dims=(voxBin, voxBin, voxBin), bounds=volBounds)
    # printHistogram(volume, logscale=False, bins=25, c='g', minbin=3)
    del apts

    ##############################################
    # Reconstruction
    write(volume, path_results + 'Limb-rec_' +
          str(Treal[t]) + '.vti', binary=False)
    write(volume.isosurface(threshold=-0.01).smoothWSinc(),
          path_results + 'Limb-rec_' + str(Treal[t]) + '.vtk', binary=False)

    del volume

    pb.print('main reconstruction ... ')

